# Example using the Tie Kernel

This kernel allows us to tie together the values of various parameters.

## Example 1: Simple RBF kernel with equal lengthscale and variance

Bit of an odd example, but simple to start with, we fix the lengthscale equal to the variance of the RBF kernel.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import GPy
import numpy as np
np.set_printoptions(precision=4,suppress=True)

X = np.arange(20)[:,None]
Y = np.sin(X)
simple = GPy.kern.RBF(input_dim=1,lengthscale=2.0,variance=1.0)

tiekern = GPy.kern.Tie(simple,1,[['.*']])
tietest = GPy.models.GPRegression(X,Y,tiekern)
tietest.checkgrad(verbose=1)
#tietest.optimize()

The above shows that the lengthscale and variance are now no longer fulfilling the gradient check. I've maybe not thought this through properly, and just took Max's suggestion to add together the gradients. We have a log likelihood, $L=log p(y|X,l,\sigma)$ and we know $\frac{\delta L}{\delta l}$ and $\frac{\delta L}{\delta \sigma}$. We are now fixing these to be equal. If we consider the infinitesimal movement from $l$ to $l+\Delta l$ and from $\sigma$ to $\sigma + \Delta \sigma$ the two $\Delta$ terms are equal. If the gradient wrt $\sigma$ at $l$ is equal to the gradient at $l + \Delta l$ then we can say that the change in $L$ between $(l,\sigma)$ is equal to the change that occurs along $l$ plus the change that occurs along $\sigma$, the upshot is we can add the two gradients together.

To achieve this in the kernel, I compute the new analytical values for the gradients, find their mean, and set all the gradients to this mean. 

One problem that appears above is the numerical approximation is calculated by adding a $\Delta l$ to the lengthscale, while the *analytical* one is effectively what happens if you add that delta to the variance too.

Below we demonstrate that by looking at how the likelihood changes when you move in both directions at the same time,

In [ ]:
print np.sum(tietest.gradient[0:2])
before = tietest.log_likelihood()
tietest.tie.rbf.variance+=0.00001
tietest.tie.rbf.lengthscale+=0.00001
after = tietest.log_likelihood()
(after-before)/0.00001

And finally the values of the kernel after optimizing, included to show that the two parameters are equal to each other now:

In [ ]:
tietest.optimize()
tietest

## Second example: For clustering

This is the motivation for this, I wanted to fit a set of models using the independent outputs kernel and the offset kernel.

Specifically I take two datasets and want to know whether they are better fitted together or seperately. Is the log likelihood increased by combining them?

On the outside is the 'Tie' kernel, which allows us to tie together different parameters.
Within this is the independent outputs kernel, this allows us to have three seperate kernels running together: The first is the combined 'Offset' kernel, which takes the two datasets and allows them to shift against one another. The other two are the datasets fitted individually.

    Tie[ MultipleOutput( Offset{ Common }, Common, Common ) ]


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import GPy
import numpy as np
np.set_printoptions(precision=4,suppress=True)

N = 10
X1 = np.arange(1,N+1,1)[:,None]
X2 = X1 + 1.2
X = np.vstack([X1,X2,X1,X2])
nans = np.empty([N*2,1])
nans[:] = np.NAN

#offset indicies
ind_offset = np.vstack([np.zeros([N,1]),np.ones([N,1]),nans])

#independent output indicies
ind_indpoutputs = np.vstack([np.zeros([N*2,1]),np.ones([N,1]),np.ones([N,1])*2])
X = np.hstack([X,ind_offset,ind_indpoutputs])
Y1 = np.sin((X[0:N,0])/10.0)[:,None]
#Y2 = np.sin((X[0:N,0])/10.0)[:,None]
Y2 = np.cos((X[0:N,0])/10.0)[:,None]
Y1 += np.random.randn(Y1.shape[0],Y1.shape[1])*0.1
Y2 += np.random.randn(Y2.shape[0],Y2.shape[1])*0.1
Y = np.vstack([Y1,Y2,Y1,Y2])

#Structure of inputs:
# actual input | offset_kernel_index | indp_output_index
#      2.4              0                     0
#      2.9              0                     0
#      3.4              1                     0
#      3.9              1                     0
#      2.4              nan                   1
#      2.9              nan                   1
#      3.4              nan                   2
#      3.9              nan                   2
#print X
#print Y

#base kernel to explain all time series with
common_kern = GPy.kern.Matern32(input_dim=1)

#the offset kernel, that can shift one time series wrt another
offset_kern = GPy.kern.Offset(common_kern,2,[0])

#we want to discourage massive offsets, which can achieve good fits by simply moving the two datasets far apart
offset_kern.offset.set_prior(GPy.priors.Gaussian(0,4.0))

#our overall kernel contains our offset kernel and two common kernels
independent_kern = GPy.kern.IndependentOutputs([offset_kern,common_kern.copy(),common_kern.copy()],index_dim=2)

tiekern = GPy.kern.Tie(independent_kern,3,[['.*lengthscale'],['.*variance']])

model = GPy.models.GPRegression(X,Y,tiekern)
model.optimize()

In [ ]:
model

The two datasets combined, with the offset applied. The plot function below doesn't show the offset occuring, so I've quickly plotted the data offset instead.

In [ ]:
model.plot(fixed_inputs=[(1,0),(2,0)],plot_data=False)
plt.plot(X2-model.tie.independ.offset.offset.values[0],Y2[0:10],'kx')
plt.plot(X1,Y1[0:10],'kx')

plt.grid()

The plot below is for the first single dataset (so ignore the unfitted rightward points)

In [ ]:
model.plot(fixed_inputs=[(1,0),(2,1)])

The plot below is for the second single dataset (so ignore the unfitted leftward points)

In [ ]:
model.plot(fixed_inputs=[(1,0),(2,2)])

In [ ]:
#The X matrix:
#
# actual input | offset_kernel_index | indp_output_index
#      2.4              0                     0
#      2.9              0                     0
#      3.4              1                     0
#      3.9              1                     0
#      2.4              nan                   1
#      2.9              nan                   1
#      3.4              nan                   2
#      3.9              nan                   2

We can take the above parameters and fit them separately in our own models. We can then get the loglikelihoods and compare if the sum of the individual models is greater or lesser than the combined model.

Notice that the hyperparameters will all now be equal, so the model complexity should be equal.

In [ ]:
n = len(X)
indepY = Y[n/2:]
indepX = X[n/2:,[0,2]]
indepX[:,1]-=1
offsetY = Y[:n/2]
offsetX = X[:n/2,[0,1]]

In [ ]:
#base kernel to explain all time series with
common_kern = GPy.kern.Matern32(input_dim=1)

#the offset kernel, that can shift one time series wrt another
offset_kern = GPy.kern.Offset(common_kern,2,[0])

#we want to discourage massive offsets, which can achieve good fits by simply moving the two datasets far apart
offset_kern.offset.set_prior(GPy.priors.Gaussian(0,4.0))

#our overall kernel contains our offset kernel and two common kernels
independent_kern = GPy.kern.IndependentOutputs([common_kern.copy(),common_kern.copy()],index_dim=1)
independent_model = GPy.models.GPRegression(indepX,indepY,independent_kern)

offset_model = GPy.models.GPRegression(offsetX,offsetY,offset_kern)

In [ ]:
independent_model.kern.Mat32.variance.fix(model.kern.independ.Mat32.variance[:])
independent_model.kern.Mat32_1.variance.fix(model.kern.independ.Mat32_1.variance[:])
independent_model.kern.Mat32.lengthscale.fix(model.kern.independ.Mat32.lengthscale[:])
independent_model.kern.Mat32_1.lengthscale.fix(model.kern.independ.Mat32_1.lengthscale[:])
independent_model.Gaussian_noise.fix(model.Gaussian_noise[:])

independent_model.update_toggle = True
independent_model.trigger_update()

offset_model.kern.Mat32.variance.fix(model.kern.independ.offset.Mat32.variance[:])
offset_model.kern.Mat32.lengthscale.fix(model.kern.independ.offset.Mat32.lengthscale[:])
offset_model.Gaussian_noise.fix(model.Gaussian_noise[:])
offset_model.kern.offset.fix(model.kern.independ.offset.offset[:])

offset_model.trigger_update()

In [ ]:
model.plot(fixed_inputs=[(1,0),(2,0)])

In [ ]:
offset_model.plot(fixed_inputs=[(1,1)])

In [ ]:
#Just checking that the independent model's likelihood is the sum of the two
#models it's made of

simple_model1 = GPy.models.GPRegression(X1,Y1,common_kern)
simple_model1.Gaussian_noise.fix(model.Gaussian_noise[:])
simple_model1.kern.variance.fix(model.kern.independ.Mat32.variance[:])
simple_model1.kern.lengthscale.fix(model.kern.independ.Mat32.lengthscale[:])
simple_model1.update_toggle = True
simple_model1.update_model()
simple_model1.plot()

simple_model2 = GPy.models.GPRegression(X2,Y2,common_kern)
simple_model2.Gaussian_noise.fix(model.Gaussian_noise[:])
simple_model2.kern.variance.fix(model.kern.independ.Mat32.variance[:])
simple_model2.kern.lengthscale.fix(model.kern.independ.Mat32.lengthscale[:])
simple_model2.update_toggle = True
simple_model2.update_model()
print simple_model1.log_likelihood() + simple_model2.log_likelihood()
print independent_model.log_likelihood()

In [ ]:
offset_model.log_likelihood()

In [ ]:
N = 5
active = []
for p in range(0,N):
    active.append([p])

diffloglikes = np.zeros([len(active),len(active)])
diffloglikes[:] = None
offsets = np.zeros([len(active),len(active)])

In [ ]:
for clusti in range(len(active)):        
    for clustj in range(clusti): #count from 0 to clustj-1
        if np.isnan(diffloglikes[clusti,clustj]):
            diffloglikes[clusti,clustj] = np.random.rand()
            offsets[clusti,clustj] = np.random.rand()

In [ ]:
top = np.unravel_index(np.nanargmax(diffloglikes), diffloglikes.shape)

In [ ]:
diffloglikes

In [ ]:
top

In [ ]:
inputs = []
data = []
for n in range(N):
    inputs.append(np.random.randn(3,1))
    data.append(np.random.randn(1,3))
if diffloglikes[top[0],top[1]]>0:
    active[top[0]].extend(active[top[1]])
    offset=offsets[top[0],top[1]]
    inputs[top[0]] = np.vstack([inputs[top[0]],inputs[top[1]]-offset])
    data[top[0]] = np.hstack([data[top[0]],data[top[1]]])
    del inputs[top[1]]
    del data[top[1]]
    del active[top[1]]
    
    #None = we need to recalculate
    diffloglikes[:,top[0]] = None
    diffloglikes[top[0],:] = None
    diffloglikes = np.delete(diffloglikes,top[1],0)
    diffloglikes = np.delete(diffloglikes,top[1],1)

In [ ]:
diffloglikes

In [ ]:
array([[      nan,     nan,     nan,     nan],
       [,     nan,     nan,     nan,     nan],
       [ ,  0.1765,     nan,     nan,     nan],
               ?         ?      ?          ?
       [   0.0623,  0.0079,  0.4415,     nan]])

In [ ]:
active

In [ ]:
inputs[2]

# Combining into a single python method

In [35]:
import matplotlib.pyplot as plt
%matplotlib inline
import GPy
import numpy as np
np.set_printoptions(precision=4,suppress=True)


def get_log_likelihood_diff(inputs,data,clusti,clustj,common_kern):
    data_i = data[clusti]
    data_j = data[clustj]
    inputs_i = inputs[clusti]
    inputs_j = inputs[clustj]
    Ni = data_i.shape[0]
    Nj = data_j.shape[0]
    N = Ni+Nj
#    print "CLUSTI, CLUSTJ"
#    print clusti, clustj
#    print "NI, NJ"
#    print Ni, Nj
#    print "INPUTS_I SHAPE"
#    print inputs_i.shape
#    print "INPUTS_J SHAPE"
#    print inputs_j.shape
    X = np.vstack([inputs_i,inputs_j,inputs_i,inputs_j])
    nans = np.empty([N,1])
    nans[:] = np.NAN
    
    #offset indicies
    ind_offset = np.vstack([np.zeros([Ni,1]),np.ones([Nj,1]),nans])

    #independent output indicies
    ind_indpoutputs = np.vstack([np.zeros([N,1]),np.ones([Ni,1]),np.ones([Nj,1])*2])
    X = np.hstack([X,ind_offset,ind_indpoutputs])
    Y1 = data_i #[:,None]
    Y2 = data_j #[:,None]
    Y = np.vstack([Y1,Y2,Y1,Y2])

    
    #these are actually the same, could equate later TODO
    indepY = np.vstack([Y1,Y2])
    indepX = np.hstack([np.vstack([inputs_i,inputs_j]),np.vstack([np.zeros([Ni,1]),np.ones([Nj,1])])])
    
    offsetY = np.vstack([Y1,Y2])
    offsetX = np.hstack([np.vstack([inputs_i,inputs_j]),np.vstack([np.zeros([Ni,1]),np.ones([Nj,1])])]) 
    
    #Structure of inputs:
    # actual input | offset_kernel_index | indp_output_index
    #      2.4              0                     0
    #      2.9              0                     0
    #      3.4              1                     0
    #      3.9              1                     0
    #      2.4              nan                   1
    #      2.9              nan                   1
    #      3.4              nan                   2
    #      3.9              nan                   2
    #print X
    #print Y

    #base kernel to explain all time series with
    common_kern = GPy.kern.Matern32(input_dim=1)

    #the offset kernel, that can shift one time series wrt another
    offset_kern = GPy.kern.Offset(common_kern,2,[0])

    #we want to discourage massive offsets, which can achieve good fits by simply moving the two datasets far apart
    offset_kern.offset.set_prior(GPy.priors.Gaussian(0,4.0))

    #our overall kernel contains our offset kernel and two common kernels
    independent_kern = GPy.kern.IndependentOutputs([offset_kern,common_kern.copy(),common_kern.copy()],index_dim=2)

    tiekern = GPy.kern.Tie(independent_kern,3,[['.*lengthscale'],['.*variance']])
    model = GPy.models.GPRegression(X,Y,tiekern)
    

    model.optimize()

    #the offset kernel, that can shift one time series wrt another
    offset_kern = GPy.kern.Offset(common_kern,2,[0])

    #we want to discourage massive offsets, which can achieve good fits by simply moving the two datasets far apart
    offset_kern.offset.set_prior(GPy.priors.Gaussian(0,4.0))

    #our overall kernel contains our offset kernel and two common kernels
    independent_kern = GPy.kern.IndependentOutputs([common_kern.copy(),common_kern.copy()],index_dim=1)
    

    
    independent_model = GPy.models.GPRegression(indepX,indepY,independent_kern)

    offset_model = GPy.models.GPRegression(offsetX,offsetY,offset_kern)
    offset = offset_model.offset[0]
    return offset_model.log_likelihood()-independent_model.log_likelihood(), offset

In [36]:
def cluster(data,inputs,common_kern):
    active = []
    for p in range(0,N):
        active.append([p])
    diffloglikes = np.zeros([len(active),len(active)])
    diffloglikes[:] = None
    offsets = np.zeros([len(active),len(active)])
    while True:
        for clusti in range(len(active)):        
            for clustj in range(clusti): #count from 0 to clustj-1
                if np.isnan(diffloglikes[clusti,clustj]):
                    diffloglikes[clusti,clustj],offsets[clusti,clustj] = get_log_likelihood_diff(inputs,data,clusti,clustj,common_kern)

        top = np.unravel_index(np.nanargmax(diffloglikes), diffloglikes.shape)

        if diffloglikes[top[0],top[1]]>0:


            active[top[0]].extend(active[top[1]])
            offset=offsets[top[0],top[1]]
            inputs[top[0]] = np.vstack([inputs[top[0]],inputs[top[1]]-offset])
            data[top[0]] = np.vstack([data[top[0]],data[top[1]]])
            del inputs[top[1]]
            del data[top[1]]
            del active[top[1]]

            #None = we need to recalculate
            diffloglikes[:,top[0]] = None
            diffloglikes[top[0],:] = None
            diffloglikes = np.delete(diffloglikes,top[1],0)
            diffloglikes = np.delete(diffloglikes,top[1],1)
        else:
            break
#        print "ACTIVE"
#        print active
#        print "DIFF LOG LIKES"
#        print diffloglikes
#        print "DATA SHAPES"
#        for d in data:
#            print d.shape
    return active

In [46]:
import sys
sys.path.append('~/GPy/GPy') #change to where your GPy is installed or remove if in path
import GPy
import numpy as np
import matplotlib.pyplot as plt
import time
import random
%matplotlib inline
#make some simulated data

S = 9 #number of time series per data point

#actual numbers of time points (0,1,2,3...)
#Note I've already removed patients with less than 3 time points.
Tpointcounts = np.array([   0.,    0.,    0.,  0.,   78.,   72.,   48.,   42.,   36., 32.,   23.,   11.,    5.,    5.,    6.,    4.,    9.,    3.,   3.,    4.,    2.,    1.,    2.,    1.,    3.,    1.,    2.,   1.,    1.,    2.,    0.,    0.,    1.,    0.,    1.,    0.,   1.,    0.,    0.])  
Tpoints = []
for i,count in enumerate(Tpointcounts):
    for j in range(int(count)):
        Tpoints.append(i)

N = 12 #len(Tpoints) #we now get N from the number of points
print("%d patients in current data set" % N)
C = 3  #number of clusters

#actual latent function
def lat_fn(t,s,c): #time, series, cluster
    return np.sin(0.01*(1.0+t)*(10.0+c)/(10.0+c)+c)+t*(0.01*c*s)+c*0.2+1-s*0.02+s+c

data = []
inputs = []
dataA = []
inputsA = []
groundtruth = []

offsets = np.random.randn(N)*0 #no time offsets

for p in range(N):
    #sample from the known distribution of number of time points.
    T = random.sample(Tpoints,1)[0]    
    Tpoints.remove(T)

    persondata = np.zeros([S,T])
    personinputs = np.zeros([T,1])
    indx = 0
    c = np.random.randint(0,C) #pick cluster we're going to put person in
    groundtruth.append(c)
    pt = 0
    for t in range(T):
        personinputs[indx,0] = pt
        indx+=1
        for s in range(S):
            persondata[s,t] = lat_fn(pt+offsets[p],s,c) + np.random.randn(1)*0.1
        pt += np.random.randint(1,10)
    data.append(persondata.T)
    inputs.append(personinputs)

12 patients in current data set


In [47]:
common_kern = GPy.kern.Matern32(input_dim=1)
cluster(data,inputs,common_kern)

[[6], [7, 5, 4, 0, 1], [11, 8, 2, 10, 9, 3]]

In [50]:
gt = np.array(groundtruth)
for idx in range(6):
    print idx,np.where(gt==idx)

0 (array([ 2,  3,  8,  9, 10, 11]),)
1 (array([6]),)
2 (array([0, 1, 4, 5, 7]),)
3 (array([], dtype=int64),)
4 (array([], dtype=int64),)
5 (array([], dtype=int64),)
